In [1]:
import os
import io
from IPython.display import Image, display, HTML
from PIL import Image
import base64 
from transformers import pipeline, AutoTokenizer

c:\Users\LENOVO\.conda\envs\gradio-env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# load the summarization pipeline
summarizer = pipeline ("summarization", model="sshleifer/distilbart-cnn-12-6", tokenizer="sshleifer/distilbart-cnn-12-6")
tokenizer = AutoTokenizer.from_pretrained("sshleifer/distilbart-cnn-12-6")

In [3]:
import json

def summarize_text(input_text):
    """Summarizes the given input text.

    Args:
        input_text (str): The text to be summarized.

    Returns:
        dict: A dictionary containing the summary under the 'summary' key.
    """

    # Tokenize and truncate input if necessary
    max_length = tokenizer.model_max_length
    inputs = tokenizer(input_text, truncation=True, max_length=max_length, return_tensors="pt")

    # Generate summary
    summary_ids = summarizer.model.generate(inputs.input_ids, max_length=50, min_length=10, do_sample=False)
    summary_text = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    # Return summary as a dictionary
    return {"summary": summary_text}

In [4]:
text = ('''The tower is 324 metres (1,063 ft) tall, about the same height
        as an 81-storey building, and the tallest structure in Paris. 
        Its base is square, measuring 125 metres (410 ft) on each side. 
        During its construction, the Eiffel Tower surpassed the Washington 
        Monument to become the tallest man-made structure in the world,
        a title it held for 41 years until the Chrysler Building
        in New York City was finished in 1930. It was the first structure 
        to reach a height of 300 metres. Due to the addition of a broadcasting 
        aerial at the top of the tower in 1957, it is now taller than the 
        Chrysler Building by 5.2 metres (17 ft). Excluding transmitters, the 
        Eiffel Tower is the second tallest free-standing structure in France 
        after the Millau Viaduct.''')

## generate the text summary
summarize_text(text)

{'summary': ' The Eiffel Tower is 324 metres (1,063 ft) tall, about the same height as an 81-storey building. It is the second tallest free-standing structure in France after the Millau Viaduct'}

In [5]:
import gradio as gr
def generate_summary(input):
    output = summarize_text(input)
    return output
    
gr.close_all()
demo = gr.Interface(fn=generate_summary, 
                        inputs=[gr.Textbox(label="Text to summarize", lines=6)], 
                        outputs=[gr.Textbox(label="Summary", lines=3)],
                        title="Text Summarization",
                        description="Summarize text using the 'shleifer/distilbart-cnn-12-6' language model.",                    
                    )
demo.launch(share=True, server_port=int(os.environ['PORT']))

Running on local URL:  http://127.0.0.1:53040
Running on public URL: https://c9798809c074dc5b7c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


In [ ]:
import gradio as gr

def summarize(input):
    output = get_completion(input)
    return output[0]['summary_text']

gr.close_all()
demo = gr.Interface(fn=summarize, 
                    inputs=[gr.Textbox(label="Text to summarize", lines=6)],
                    outputs=[gr.Textbox(label="Result", lines=3)],
                    title="Text summarization with distilbart-cnn",
                    description="Summarize any text using the `shleifer/distilbart-cnn-12-6` model under the hood!"
                   )
demo.launch(share=True, server_port=int(os.environ['PORT2']))